# Control Package tutorials

In [ ]:
from IPython.display import SVG, display

## Transfer function

Find the Transfer Function of the following Spring-Mass dampering system :

In [ ]:
display(SVG(filename="images/Control_Problems_Q2.svg"))

In [ ]:
# Imports
from sympy import (
    Function, 
    laplace_transform, laplace_initial_conds, laplace_correspondence,
    diff, Symbol, solve
)
from sympy.physics.control import TransferFunction

In [ ]:
from sympy.abc import s, t # this is equivalent to s = Symbol('s'), t = Symbol('t')
y = Function('y')
Y = Function('Y')
u = Function('u')
U = Function('U')
k = Symbol('k') # Spring Constant
c = Symbol('c') # Damper
m = Symbol('m') # Mass of block

Given the differential equation, you can use the `laplace_transform` function from SymPy to convert the equation from the time domain to the Laplace domain. The argument `noconds=True` specifies that the Laplace transform should be computed without including terms for initial conditions, resulting in a purely algebraic expression in the Laplace domain. This is useful when you want to analyze the system's behavior without considering specific initial states.

In [ ]:
f = m*diff(y(t), t, t) + c*diff(y(t), t) + k*y(t) - u(t)
F = laplace_transform(f, t, s, noconds=True)

F

Replace time-domain functions with Laplace-domain symbols, so you get an expression entirely in the Laplace domain (algebraic form).

In [ ]:
F = laplace_correspondence(F, {u: U, y: Y})
F

Apply initial conditions

In [ ]:
F = laplace_initial_conds(F, t, {y: [0, 0]})
F

Solve for the Transfer Function Y(s)/U(s)

In [ ]:
t = (solve(F, Y(s))[0])/U(s) # To construct Transfer Function from Y(s) and U(s)
print(type(t))
t

In [ ]:
tf = TransferFunction.from_rational_expression(t, s)
print(type(tf))
tf

## Equivalent transfer function of an interconnected system

In [ ]:
display(SVG(filename="images/Control_Problems_Q5.svg"))

Given the following transfer functions:

$
\begin{align}
G1 &= \frac{1}{10 + s} \\
G2 &= \frac{1}{1 + s} \\
G3 &= \frac{1 + s^2}{4 + 4s + s^2} \\
G4 &= \frac{1 + s}{6 + s} \\
H1 &= \frac{1 + s}{2 + s} \\
H2 &= \frac{2(6 + s)}{1 + s} \\
H3 &= 1
\end{align}
$

Where $s$ is variable of the transfer function in the Laplace domain.

Find: 
1. The equivalent transfer function representing the system given above.
2. Pole-Zero of the system.

In [ ]:
from sympy.abc import s
from sympy.physics.control import *

First, we define the transfer function of each block in the interconnected system. Each block (G1, G2, G3, G4, H1, H2, H3) is represented as a transfer function in the Laplace domain using the variable \( s \). This allows us to model the dynamic behavior of each subsystem before combining them to analyze the overall system.

In [ ]:
G1 = TransferFunction(1, 10 + s, s)
G2 = TransferFunction(1, 1 + s, s)
G3 = TransferFunction(1 + s**2, 4 + 4*s + s**2, s)
G4 = TransferFunction(1 + s, 6 + s, s)
H1 = TransferFunction(1 + s, 2 + s, s)
H2 = TransferFunction(2*(6 + s), 1 + s, s)
H3 = TransferFunction(1, 1, s)


To analyze the overall system, we start by defining the connections between the individual transfer function blocks using series and feedback interconnections. The `doit()` method in SymPy's control toolbox is used to explicitly compute and simplify the result of these interconnections, reducing composite system objects (like `Series` or `Feedback`) to a single `TransferFunction` that represents the equivalent transfer function of the interconnected system. This step is essential for further analysis, such as finding poles, zeros, or plotting system responses.

In [ ]:
sys1 = Series(G3, G4)
sys2 = Feedback(sys1, H1, 1).doit()
sys3 = Series(G2, sys2)
sys4 = Feedback(sys3, H2).doit()
sys5 = Series(G1, sys4)
sys6 = Feedback(sys5, H3)
sys6

In [ ]:
sys6 = sys6.doit(cancel=True, expand=True)
sys6

In [ ]:
pole_zero_plot(sys6)

## Mechanics Problems using StateSpace

### Problem description: Find the frequency response of the system

In [ ]:
display(SVG(filename="images/Mechanics_Problems_Q1.svg"))

A spring-mass-damping system can be modeled using a mass (m), a spring with a constant (k), and a damper with a damping coefficient (b). The spring force is proportional to the displacement of the mass, and the damping force is proportional to the velocity of the mass.

The equation of motion for the mass-spring-damper system is given by:

$m * \ddot{x} + b * \dot{x} + k x = F(t)$

where:

- $x$ is the displacement of the mass,
- $\dot{x}$ is the velocity of the mass,
- $\ddot{x}$ is the acceleration of the mass,
- $F(t)$ is the external force applied to the system.

The state-space can be represented by:

$
\begin{align}
A = \begin{bmatrix}
0 & 1 \\
-\frac{k}{m} & -\frac{b}{m} \\
\end{bmatrix}, \quad
B = \begin{bmatrix}
0 \\
\frac{1}{m} \\
\end{bmatrix}, \quad
C = \begin{bmatrix}
1 & 0 \\
\end{bmatrix}, \quad
D = \begin{bmatrix}
0 \\
\end{bmatrix}
\end{align}
$

$
\begin{align}
\dot{x} &= A x + B F(t) \\
y &= C x + D F(t)
\end{align}
$

Using SymPy’s Control Systems Toolbox (CST), we can define the state-space representation and convert it to the transfer function.

### Solution

In [ ]:
from sympy import symbols, Matrix
from sympy.physics.control import StateSpace, TransferFunction

Define the variables

In [ ]:
m, k, b = symbols('m k b')

Define the state-space matrices

In [ ]:
A = Matrix([[0, 1], [-k/m, -b/m]])
B = Matrix([[0], [1/m]])
C = Matrix([[1, 0]])
D = Matrix([[0]])

Create the StateSpace model

In [ ]:
ss = StateSpace(A, B, C, D)
ss

Converting StateSpace to TransferFunction by rewrite method.

In [ ]:
tf = ss.rewrite(TransferFunction)[0][0]
tf

In [ ]:
tf = TransferFunction(A, B, C, D)

### Plots

We will visualize the system's characteristics using several standard plots:

- **Pole-Zero Plot:** Shows the locations of poles and zeros of the transfer function in the complex plane, providing insight into system stability and response.
- **Step Response Plot:** Illustrates how the system responds to a step input, revealing transient and steady-state behavior.
- **Bode Magnitude Plot:** Displays the frequency response magnitude, helping analyze how the system amplifies or attenuates signals at different frequencies.
- **Bode Phase Plot:** Shows the phase shift introduced by the system across frequencies, important for understanding system dynamics and stability margins.

In [ ]:
from sympy.physics.control import (
    pole_zero_plot,
    step_response_plot,
    bode_magnitude_plot,
    bode_phase_plot
)

Before using plotting or system response functions, it is necessary to substitute the symbolic parameters ($m$, $b$, $k$) with numerical values. This is because visualization functions require a transfer function with numeric coefficients in order to generate the plots correctly. For example, you can use the `.subs()` method to replace the symbols with specific values, as done in the `tf_num` variable.

In [ ]:
step_response_plot(tf)

In [ ]:
tf_num = tf.subs({m: 1, b: 2, k: 3})
tf_num

In [ ]:
step_response_plot(tf_num)

In [ ]:
pole_zero_plot(tf_num)

In [ ]:
bode_magnitude_plot(tf_num)

In [ ]:
bode_phase_plot(tf_num)